# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [23]:
# Write your code here
import nltk
import pandas as pd
import csv
import spacy
import io as stringIOModule
import re
import bs4
from bs4 import BeautifulSoup as soup
from sklearn.feature_extraction.text import CountVectorizer
from urllib.request import urlopen as uReq

#nltk.download('punkt')
#nltk.download('stopwords')

data = pd.read_csv('/posTag.csv', low_memory = False)
print(data)




    Cleaned_title
0        abstract
1           found
2         describ
3          method
4         statist
..            ...
219       languag
220       process
221           nlp
222       concern
223       develop

[224 rows x 1 columns]


In [14]:
word_vectorizer = CountVectorizer(stop_words=None)
sparse_matrix = word_vectorizer.fit_transform(data['Cleaned_title'].values)
frequencies = sum(sparse_matrix).toarray()[0]
df = pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['Frequency'])
print(df)

            Frequency
abil                1
abl                 1
abstract            2
access              1
accomplish          1
...               ...
versatil            1
virtual             1
way                 3
work                1
yet                 1

[129 rows x 1 columns]


In [16]:

with open('/posTag.csv') as a:
  read = csv.reader(a)
  next(read, None)
  Cleaned_title = [row[0] for row in read]
with open('posTag.txt', mode="w") as o:
  for i in Cleaned_title:
    o.write("%s\n" % i)
def bigram(file):
  l = []
  b = {}
  u = {}
  t = open(file, 'r').read()
  l = t.strip().split()
  del t
  for f in l:
    if not f in u:
      u[f] = 1
    else:
      u[f] += 1
  for p in range(len(l) - 1):
    temp = (l[p], l[p+1])
    if not temp in b:
      b[temp] = 1
    else:
      b[temp] += 1
  print('Generated', len(b), 'bigrams')
  total = sum(u.values())
  for n,t in b.items():
    first_word = n[0]
    first_wordcount = u[first_word]
    bi = {}
    bi = b[n]/u[first_word]
    if t == 2:
      print(n[0],n[1],t,bi)
bigram('posTag.txt')

Generated 193 bigrams
maximum entropi 2 1.0
field natur 2 1.0
term condit 2 1.0
process nlp 2 0.16666666666666666
descript logic 2 1.0
syntact semant 2 1.0
semant interpret 2 0.4


In [17]:
nlp = spacy.load("en")
filename = open("posTag.txt")
d = nlp(filename.read())
noun = []
for p in d.noun_chunks:
  noun.append(p.text)
df = pd.DataFrame(noun, columns=['noun'])
word_vectorizer = CountVectorizer(ngram_range=(3,3), analyzer='word')
sparse_matrix = word_vectorizer.fit_transform(df['noun'].values.astype('U'))
frequencies = sum(sparse_matrix).toarray()[0]
df1 = pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(),columns=['Frequency'])
df1['NounProbabilities'] = df1/df1.max()
print(noun)
df1

['abstract', 'describ\nmethod', 'statist', 'model\nbase', 'maximum', 'entropi', 'maximumlikelihood', 'approach\nautomat', 'maximum', 'entropi\nmodel', 'approach', 'effici', 'sever', 'problem', 'natur\nlanguag\nprocess\nscale', 'condit', 'field', 'natur', 'languag\nprocess\nterm', 'condit\nterm', 'condit', 'copyright\nwork', 'deposit', 'minerva', 'access', 'retain\npaper', 'address', 'issu', 'cooper', 'linguist', 'languag\nprocess', 'nlp', 'gener', 'linguist', 'machin', 'translat', 'mt', 'focus', 'cooper\nname', 'applic', 'linguist', 'nlp', 'natur', 'languag\nprocess', 'descript', 'encod', 'knowledg\nbase', 'syntact', 'semant', 'pragmat', 'element', 'semant', 'interpret', 'natur', 'languag', 'gener\nprocess', 'descript\nlogic', 'propos\nunifi', 'neural', 'network\narchitectur', 'algorithm', 'variou', 'natur\nlanguag\nprocess', 'task', 'partofspeech', 'tag', 'chunk', 'name', 'entiti', 'recognit\nsemant', 'role', 'label', 'versatil', 'achiev', 'tri', 'task', 'natur\nlanguag\nprocess', 'na

,Frequency,NounProbabilities
languag process scale,1,0.2
languag process term,1,0.2
natur languag process,5,1.0
sens cover process,1,0.2


# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [19]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords 
data = pd.read_csv("/posTag.csv")
tfidf2 = data.dropna()
tfidf1 = (tfidf2['Cleaned_title'].apply(lambda z:pd.value_counts(z.split(" "))).sum(axis=0).reset_index())
tfidf1.columns = ['words','tf']
for l , word in enumerate(tfidf1['words']):
  tfidf1.loc[l,'idf'] = np.log(data.shape[0]/(len(tfidf2[tfidf2['Cleaned_title'].str.contains(word)])))
tfidf1['tf*idf'] = tfidf1['tf'] * tfidf1['idf']
tfidf1

,words,tf,idf,tf*idf
0,abstract,2.0,4.718499,9.436998
1,found,1.0,5.411646,5.411646
2,describ,2.0,4.718499,9.436998
3,method,1.0,5.411646,5.411646
4,statist,1.0,5.411646,5.411646
...,...,...,...,...
124,mean,1.0,5.411646,5.411646
125,abl,1.0,5.411646,5.411646
126,peopl,1.0,5.411646,5.411646
127,concern,1.0,5.411646,5.411646


In [27]:
import numpy.linalg as alg
from sklearn.metrics.pairwise import cosine_similarity

train = tfidf2['Cleaned_title'].values.tolist()
test = ["real world problems for concepts in statistical estimation and recognition of pattern"]
stop = stopwords.words('english')
vectorizer = CountVectorizer(stop_words = stop)
transformer = TfidfTransformer()
array = vectorizer.fit_transform(train).toarray()
tarray = vectorizer.transform(test).toarray()
ab = lambda c,d : np.inner(c,d)/(alg.norm(c)*alg.norm(d))
result = []
for x in array:
  for v in tarray:
    cosine = ab(x, v)
    result.append(cosine)
d = tfidf2.filter(['No','Cleaned_title'], axis=1)
s = pd.Series(result)
d['Cosine_similarity'] = s.values
d.drop(d.loc[d['Cosine_similarity']==0].index, inplace=True)
d["Rank"] = d["Cosine_similarity"].rank().astype(float)
d.sort_values("Cosine_similarity", inplace=True)
d

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


,Cleaned_title,Cosine_similarity,Rank
0,abstract,NaN,NaN
1,found,NaN,NaN
2,describ,NaN,NaN
3,method,NaN,NaN
4,statist,NaN,NaN
...,...,...,...
219,languag,NaN,NaN
220,process,NaN,NaN
221,nlp,NaN,NaN
222,concern,NaN,NaN


# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [ ]:
# The GitHub link of your final csv file

# Link: https://github.com/Aakansha06/In-class-exercise1/blob/master/Sentiment%20Analysis.csv